<a href="https://colab.research.google.com/github/germfsip/2132ReconocimientoPatrones/blob/main/Clase8RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reconocimiento de Patrones
##Dr. Gerardo Martinez
##Lección 8 - Búsqueda de hiperparámetros

## Presentación de los Datos

In [33]:
import numpy as np
import pandas as pd
import time
import warnings
from sklearn import metrics, preprocessing, tree
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore")



In [34]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('loan_prediction.csv')
df.head()

Saving loan_prediction.csv to loan_prediction (2).csv


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,0.0,360.0,1.0,1
1,4583,1508.0,128.0,360.0,1.0,0
2,3000,0.0,66.0,360.0,1.0,1
3,2583,2358.0,120.0,360.0,1.0,1
4,6000,0.0,141.0,360.0,1.0,1


In [35]:
from sklearn.tree import DecisionTreeClassifier as dt
model = dt()

## Train-Test Split

In [36]:
x = np.asanyarray(df.drop(columns=['Loan_Status']))
y = np.asanyarray(df[['Loan_Status']])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
model = dt(min_samples_leaf=4)
model.fit(xtrain, ytrain)
print('Train: ', model.score(xtrain, ytrain))
print('Test: ', model.score(xtest, ytest))

ypred = model.predict(xtest)
print('Accuracy Score : ' + str(accuracy_score(ytest,ypred)))
print('Precision Score : ' + str(precision_score(ytest,ypred)))
print('Recall Score : ' + str(recall_score(ytest,ypred)))
print('F1 Score : ' + str(f1_score(ytest,ypred)))

#imprimir matriz de confusion
print('Confusion Matrix : \n' + str(confusion_matrix(ytest,ypred)))


Train:  0.8574338085539714
Test:  0.7073170731707317
Accuracy Score : 0.7073170731707317
Precision Score : 0.875
Recall Score : 0.7
F1 Score : 0.7777777777777777
Confusion Matrix : 
[[24  9]
 [27 63]]


## Validación cruzada

In [37]:
model = dt()
scores = cross_val_score(model, xtrain, ytrain, cv=64, scoring='f1_macro')
print(scores)

[0.73333333 0.66666667 0.38461538 0.46666667 0.33333333 0.73333333
 0.56363636 1.         0.46666667 0.66666667 0.66666667 0.56363636
 0.56363636 0.56363636 0.73333333 0.46666667 0.87301587 0.61904762
 0.61904762 0.33333333 0.73333333 0.73333333 0.56363636 0.87301587
 0.61904762 0.66666667 0.46666667 0.56363636 0.66666667 0.61904762
 0.56363636 0.73333333 0.61904762 0.73333333 0.56363636 0.73333333
 0.73333333 0.66666667 0.66666667 1.         0.56363636 0.27272727
 0.46666667 0.53333333 0.65       0.53333333 0.41666667 0.3
 0.57142857 0.65       0.78787879 0.65       0.65       0.3
 0.36363636 0.41666667 0.65       0.78787879 0.3        0.65
 1.         0.78787879 0.41666667 0.65      ]


In [38]:
print(scores.mean(), scores.std())


0.607865225052725 0.16650604780718214


## KFold

In [39]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

train_scores = []
dev_scores = []

for train_index, test_index in kf.split(xtrain):
  train, dev = xtrain[train_index], xtrain[test_index]
  y_train, y_dev = ytrain[train_index], ytrain[test_index]
  model = dt()
  model.fit(train, y_train)
  train_scores.append(model.score(train,y_train))
  dev_scores.append(model.score(dev, y_dev))


print(train_scores)
print(np.mean(train_scores))
print(dev_scores)
print(np.mean(dev_scores))

ypred = model.predict(xtest)
print('Accuracy Score : ' + str(accuracy_score(ytest,ypred)))
print('Precision Score : ' + str(precision_score(ytest,ypred)))
print('Recall Score : ' + str(recall_score(ytest,ypred)))
print('F1 Score : ' + str(f1_score(ytest,ypred)))

#imprimir matriz de confusion
print('Confusion Matrix : \n' + str(confusion_matrix(ytest,ypred)))


[1.0, 1.0, 1.0, 1.0, 1.0]
1.0
[0.7171717171717171, 0.673469387755102, 0.6020408163265306, 0.6530612244897959, 0.5816326530612245]
0.645475159760874
Accuracy Score : 0.7073170731707317
Precision Score : 0.8068181818181818
Recall Score : 0.7888888888888889
F1 Score : 0.797752808988764
Confusion Matrix : 
[[16 17]
 [19 71]]


## Grid Search

In [40]:
parameters = {'max_depth':[1,2,3,4,5],
              'min_samples_leaf':[1,2,3,4,5],
              'min_samples_split':[2,3,4,5],
              'criterion' : ['gini','entropy']}

In [41]:
search_obj = GridSearchCV(model, parameters, cv=5, scoring='f1_macro')
fit_obj = search_obj.fit(x, y)
print(fit_obj.cv_results_['mean_test_score'])
best_model = fit_obj.best_estimator_

[0.70741357 0.70741357 0.70741357 0.70741357 0.70741357 0.70741357
 0.70741357 0.70741357 0.70741357 0.70741357 0.70741357 0.70741357
 0.70741357 0.70741357 0.70741357 0.70741357 0.70741357 0.70741357
 0.70741357 0.70741357 0.686559   0.686559   0.686559   0.686559
 0.686559   0.686559   0.686559   0.686559   0.68503533 0.68503533
 0.68503533 0.68503533 0.68503533 0.68503533 0.68736277 0.68503533
 0.6880986  0.6880986  0.6880986  0.6880986  0.686559   0.686559
 0.686559   0.686559   0.686559   0.686559   0.686559   0.686559
 0.68586884 0.68586884 0.68843179 0.68586884 0.68843179 0.68586884
 0.68586884 0.69075923 0.67941612 0.67941612 0.67941612 0.67941612
 0.68005905 0.68005905 0.68005905 0.68005905 0.67976772 0.68005905
 0.68285165 0.68005905 0.67575011 0.68034984 0.67726496 0.68034984
 0.68354296 0.68202684 0.68202684 0.68121553 0.66985454 0.66985454
 0.66985454 0.67366259 0.65961007 0.66358862 0.66210385 0.66063377
 0.66458633 0.66596589 0.66452103 0.656273   0.65982071 0.6478214
 0

In [42]:
print(best_model)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=1, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [43]:
best_model.fit(xtrain, ytrain)
# Make predictions using the new model.
print('Train: ', best_model.score(xtrain, ytrain))
print('Test: ', best_model.score(xtest, ytest))

ypred = best_model.predict(xtest)
print('Accuracy Score : ' + str(accuracy_score(ytest,ypred)))
print('Precision Score : ' + str(precision_score(ytest,ypred)))
print('Recall Score : ' + str(recall_score(ytest,ypred)))
print('F1 Score : ' + str(f1_score(ytest,ypred)))

#imprimir matriz de confusion
print('Confusion Matrix : \n' + str(confusion_matrix(ytest,ypred)))


Train:  0.7617107942973523
Test:  0.8048780487804879
Accuracy Score : 0.8048780487804879
Precision Score : 0.8235294117647058
Recall Score : 0.9333333333333333
F1 Score : 0.8749999999999999
Confusion Matrix : 
[[15 18]
 [ 6 84]]


## Ramdonized Search

In [44]:
search_obj = RandomizedSearchCV(model, parameters, cv=5, 
                                scoring='f1_macro', n_iter=15)
fit_obj = search_obj.fit(xtrain, ytrain)
print(fit_obj.cv_results_['mean_test_score'])
best_model = fit_obj.best_estimator_

[0.67970961 0.70384919 0.67050443 0.64742572 0.66594611 0.70384919
 0.67467443 0.70188663 0.675087   0.67658326 0.67386613 0.70030196
 0.68072048 0.67509626 0.70384919]


In [45]:
best_model.fit(xtrain, ytrain)
# Make predictions using the new model.
print('Train: ', best_model.score(xtrain, ytrain))
print('Test: ', best_model.score(xtest, ytest))
ypred = best_model.predict(xtest)
print('Accuracy Score : ' + str(accuracy_score(ytest,ypred)))
print('Precision Score : ' + str(precision_score(ytest,ypred)))
print('Recall Score : ' + str(recall_score(ytest,ypred)))
print('F1 Score : ' + str(f1_score(ytest,ypred)))

#imprimir matriz de confusion
print('Confusion Matrix : \n' + str(confusion_matrix(ytest,ypred)))

Train:  0.769857433808554
Test:  0.7967479674796748
Accuracy Score : 0.7967479674796748
Precision Score : 0.8282828282828283
Recall Score : 0.9111111111111111
F1 Score : 0.8677248677248678
Confusion Matrix : 
[[16 17]
 [ 8 82]]
